In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
import cv2
import copy
import os
import numpy as np

def imgshow(img):
    if len(img.shape) == 3:
        plt.imshow(img[:, :, ::-1])
    else:
        plt.imshow(img,cmap='gray',vmin=0, vmax=255)
    plt.show() 


In [ ]:
def mkdir_always(path):
    if not os.path.isdir(path):
        os.mkdir(path)

In [ ]:
CHESSBOARD_SIZE = (6, 9)


def calibrate(chessboard_path, show_chessboard=False):
    # Logical coordinates of chessboard corners
    obj_p = np.zeros((1, CHESSBOARD_SIZE[0] * CHESSBOARD_SIZE[1], 3), np.float32)
    obj_p[0, :, :2] = np.mgrid[0:CHESSBOARD_SIZE[0], 0:CHESSBOARD_SIZE[1]].T.reshape(-1, 2)

    obj_points = []     # 3d point in real world space
    img_points = []     # 2d points in image plane.

    # Iterate through all images in the folder
    dirs = os.listdir(chessboard_path)
    image_list = [x for x in dirs if os.path.isfile(os.path.join(chessboard_path, x))]
    gray = None
    for image in image_list:
        img = cv2.imread(os.path.join(chessboard_path, image))
        #img = cv2.resize(img, (320,240), interpolation=cv2.INTER_AREA)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, CHESSBOARD_SIZE,
                                                 cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
        if ret:
            # Refining corners position with sub-pixels based algorithm
            obj_points.append(obj_p)
            cv2.cornerSubPix(gray, corners, (3, 3), (-1, -1),
                             (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01))
            img_points.append(corners)
            print('Image is valid for calibration:' + image )
            if show_chessboard:
                cv2.drawChessboardCorners(img, CHESSBOARD_SIZE, corners, ret)
                cv2.imwrite(os.path.join('./Chessboards_Corners', image), img)
        else:
            print('Image is NOT valid for calibration:' + image )

#     print(obj_points)  
#     print(img_points)   
            
    k = np.zeros((3, 3))
    d = np.zeros((4, 1))
    dims = gray.shape[::-1]
    num_valid_img = len(obj_points)
    if num_valid_img > 0:
        rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for _ in range(num_valid_img)]
        tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for _ in range(num_valid_img)]
        rms, _, _, _, _ = cv2.fisheye.calibrate(obj_points, img_points, gray.shape[::-1], k, d, rvecs, tvecs,
                                                # cv2.fisheye.CALIB_CHECK_COND +
                                                # When CALIB_CHECK_COND is set, the algorithm checks if the detected corners of each images are valid.
                                                # If not, an exception is thrown which indicates the zero-based index of the invalid image.
                                                # Such image should be replaced or removed from the calibration dataset to ensure a good calibration.
                                                cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC +
                                                cv2.fisheye.CALIB_FIX_SKEW +
                                                cv2.fisheye.CALIB_FIX_PRINCIPAL_POINT,
                                                (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6))
    print("Found " + str(num_valid_img) + " valid images for calibration")
    return k, d, dims



if not os.path.exists('./parameters'):
    os.makedirs('./parameters')
if not os.path.exists('./Chessboards_Corners'):
    os.makedirs('./Chessboards_Corners')

K, D, Dims = calibrate('./Chessboards', show_chessboard=True)
np.save('./parameters/Dims', np.array(Dims))
np.save('./parameters/K', np.array(K))
np.save('./parameters/D', np.array(D))

In [ ]:
def calc_unfish_map(k, d, dims,alpha=0.5):  
    nk = k.copy()
   
    # #enlarge image twice to get corner info
    #nk[0,0]=k[0,0]/2
    #nk[1,1]=k[1,1]/2
    
#     nk[0,0]=k[0,0]/1.25
#     nk[1,1]=k[1,1]/1.25
    
#     nk, roi = cv2.getOptimalNewCameraMatrix(k, d, dims, 0,centerPrincipalPoint = True)

    nk, roi = cv2.getOptimalNewCameraMatrix(k, d, dims, alpha, dims,centerPrincipalPoint = True)
    
    print(k)
    print(nk)
        
        
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(k, d, np.eye(3), nk, dims, cv2.CV_16SC2)
    
    map1,map2 = cv2.convertMaps(map1, map2, cv2.CV_16SC2)


    return map1,map2

In [ ]:
Dims = tuple(np.load('./parameters/Dims.npy'))
K = np.load('./parameters/K.npy')
D = np.load('./parameters/D.npy')

w = Dims[0]
h = Dims[1]

unfishmap1,unfishmap2=calc_unfish_map(k=K, d=D, dims=Dims,alpha=0.6)

# print(K)
# print(D)


In [ ]:
# print(unfishmap1[::,::,0])

center_x = (w-1)/2
imgshow( abs(unfishmap1[::,::,0]-center_x)/center_x*255  )
center_y = (h-1)/2
imgshow( abs(unfishmap1[::,::,1]-center_y)/center_y*255  )

In [ ]:
x_curve = np.linspace(-center_x/1.25,center_x/1.25,640)+center_x

s = (h,w,2)

map1 = np.zeros(s,dtype=np.float32)
map1[::,::,0] = x_curve


img = abs(unfishmap1[::,::,0]-map1[::,::,0])

plt.imshow(img,cmap='gray')

print(img)


In [ ]:
imgs = [cv2.imread('../angle_screenshots/img_{}.png'.format(i*10)) for i in range(0,10+1)]



for img in imgs:
    imgshow(img)
    img = cv2.remap(img, unfishmap1, unfishmap2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT) 
    imgshow(img)



In [ ]:
imgs = [cv2.imread('../angle_screenshots/img_{}.png'.format(i*10)) for i in range(0,10+1)]



for img in imgs:
    imgshow(img)
    img = cv2.remap(img, unfishmap1, unfishmap2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT) 
    imgshow(img)
